In [1]:
import yaml
from pathlib import Path
from ese.experiment.analysis.analysis_utils.inference_utils import preload_calibration_metrics

code_root = Path("/storage/vbutoi/projects/ESE")
scratch_root = Path("/storage/vbutoi/scratch/ESE/inference/02_19_24_WMH_SmoothNectar")

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
%%yaml cal_cfg 

global_calibration:
    num_prob_bins: 15
    num_classes: 2
    neighborhood_width: 3
    square_diff: False 

<IPython.core.display.Javascript object>

In [3]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"
##################################################
with open(inf_cfg_root / "Debug_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

cal_metrics_cfg["global_cal_metrics"] = preload_calibration_metrics(
    base_cal_cfg=cal_cfg["global_calibration"],
    cal_metrics_dict=cal_metrics_cfg["global_cal_metrics"]
)

## Pick the product of the probs. 

In [4]:
import pickle 

def load_pkl(f_name):
    with open(f_name, 'rb') as f:
       return pickle.load(f)

In [5]:
ind_tl_uncalibrated_dir = scratch_root / "WMH_Individual_Uncalibrated" / "20240219_104110-WJRG-edd491052db5c8a6e51df561e715e271" / "tl_pixel_meter_dict.pkl"
ind_tl_nectar_scaling_dir = scratch_root / "WMH_Individual_NectarScaling" / "20240218_111136-SAL0-ee42c17246fe42ced961b717f000d4a9" / "tl_pixel_meter_dict.pkl"
ind_tl_hard_nectar_binning_dir = scratch_root / "WMH_Individual_NectarBinning" / "20240219_123236-QKA7-11bdebd0a8159f669a5aa7cac80ca92f" / "tl_pixel_meter_dict.pkl"
ind_tl_soft_nectar_binning_dir = scratch_root / "WMH_Individual_SoftNectarBinning" / "20240219_123301-3E9G-70144130c84b525b784f83f67e37b067"/ "tl_pixel_meter_dict.pkl" 

ind_classwise_uncalibrated_dir = scratch_root / "WMH_Individual_Uncalibrated" / "20240219_104110-WJRG-edd491052db5c8a6e51df561e715e271" / "cw_pixel_meter_dict.pkl"
ind_classwise_nectar_scaling_dir = scratch_root / "WMH_Individual_NectarScaling" / "20240218_111136-SAL0-ee42c17246fe42ced961b717f000d4a9" / "cw_pixel_meter_dict.pkl"
ind_classwise_hard_nectar_binning_dir = scratch_root / "WMH_Individual_NectarBinning" / "20240219_123236-QKA7-11bdebd0a8159f669a5aa7cac80ca92f" / "cw_pixel_meter_dict.pkl"
ind_classwise_soft_nectar_binning_dir = scratch_root / "WMH_Individual_SoftNectarBinning" / "20240219_123301-3E9G-70144130c84b525b784f83f67e37b067"/ "cw_pixel_meter_dict.pkl" 

In [6]:
ind_calibrator_dict = {
    "Uncalibrated": {
        "top_label": load_pkl(ind_tl_uncalibrated_dir),
        "classwise": load_pkl(ind_classwise_uncalibrated_dir)
    },
    "NectarScaling": {
        "top_label": load_pkl(ind_tl_nectar_scaling_dir),
        "classwise": load_pkl(ind_classwise_nectar_scaling_dir)
    },
    "HardNectarBinning": {
        "top_label": load_pkl(ind_tl_hard_nectar_binning_dir),
        "classwise": load_pkl(ind_classwise_hard_nectar_binning_dir)
    },
    "SoftNectarBinning": {
        "top_label": load_pkl(ind_tl_soft_nectar_binning_dir),
        "classwise": load_pkl(ind_classwise_soft_nectar_binning_dir)
    },
}

In [7]:
# # Loop through the calibration metrics and add them to the dataframe.
# for cal_metric_name, cal_metric_dict in cal_metrics_cfg["global_cal_metrics"].items():
#     for calibrator in ind_calibrator_dict:
#         for split in ["cal"]:
#             cal_type = cal_metric_dict['cal_type']
#             if cal_type in ["classwise", "toplabel"]:
#                 print(f"CALIBRATOR:{calibrator}")
#                 if cal_type == 'classwise':
#                     cal_info = cal_metric_dict['_fn'](
#                         pixel_meters_dict=ind_calibrator_dict[calibrator]["classwise"][split],
#                         return_dict=True
#                     )
#                 else:
#                     cal_info = cal_metric_dict['_fn'](
#                         pixel_meters_dict=ind_calibrator_dict[calibrator]["top_label"][split],
#                         return_dict=True
#                     )
#                 print(f"Global Bin counts: (Total: {cal_info['bin_amounts'].sum()})")
#                 print(cal_info["bin_amounts"])
#                 print()
#                 print("Global Freqs:")
#                 print(cal_info["bin_freqs"])
#                 print()
#                 print("Global Confs:")
#                 print(cal_info["bin_confs"])
#                 print()
#                 print("Global Bin cal errors:")
#                 print(cal_info["bin_cal_errors"])
#                 print()
#                 print("SPLIT:{} | CALIBRATION METRIC:{} | LOSS:{}".format(
#                     split, cal_metric_name, cal_info['cal_error']
#                 ))
#                 print()
#                 print()
#             else:
#                 raise ValueError(f"Calibration type {cal_metric_dict['cal_type']} not recognized.")

## Looking at the Ensemble Methods.

In [8]:
ens_tl_uncalibrated_dir = scratch_root / "WMH_Ensemble_Uncalibrated" / "20240219_104201-BKCC-a7f56d80c4bf65f817a1cb4dc10d85b2" / "tl_pixel_meter_dict.pkl"
ens_tl_nectar_scaling_dir = scratch_root / "WMH_Ensemble_NectarScaling" / "20240218_111255-UBCH-995007dddba34d84eb7ae0aa6406373c" / "tl_pixel_meter_dict.pkl"
ens_tl_hard_nectar_binning_dir = scratch_root / "WMH_Ensemble_NectarBinning" / "20240219_123424-J8FG-669764637a558a02fa9f296ad0470121" / "tl_pixel_meter_dict.pkl"
ens_tl_soft_nectar_binning_dir = scratch_root / "WMH_Ensemble_SoftNectarBinning" / "20240219_123443-KR38-006b299497cfbed50526e828ad1d123a"/ "tl_pixel_meter_dict.pkl" 

ens_classwise_uncalibrated_dir = scratch_root / "WMH_Ensemble_Uncalibrated" / "20240219_104201-BKCC-a7f56d80c4bf65f817a1cb4dc10d85b2" / "cw_pixel_meter_dict.pkl"
ens_classwise_nectar_scaling_dir = scratch_root / "WMH_Ensemble_NectarScaling" / "20240218_111255-UBCH-995007dddba34d84eb7ae0aa6406373c" / "cw_pixel_meter_dict.pkl"
ens_classwise_hard_nectar_binning_dir = scratch_root / "WMH_Ensemble_NectarBinning" / "20240219_123424-J8FG-669764637a558a02fa9f296ad0470121" / "cw_pixel_meter_dict.pkl"
ens_classwise_soft_nectar_binning_dir = scratch_root / "WMH_Ensemble_SoftNectarBinning" / "20240219_123443-KR38-006b299497cfbed50526e828ad1d123a"/ "cw_pixel_meter_dict.pkl" 

In [9]:
ensemble_calibrator_dict = {
    "Uncalibrated": {
        "top_label": load_pkl(ens_tl_uncalibrated_dir),
        "classwise": load_pkl(ens_classwise_uncalibrated_dir)
    },
    "NectarScaling": {
        "top_label": load_pkl(ens_tl_nectar_scaling_dir),
        "classwise": load_pkl(ens_classwise_nectar_scaling_dir)
    },
    "HardNectarBinning": {
        "top_label": load_pkl(ens_tl_hard_nectar_binning_dir),
        "classwise": load_pkl(ens_classwise_hard_nectar_binning_dir)
    },
    "SoftNectarBinning": {
        "top_label": load_pkl(ens_tl_soft_nectar_binning_dir),
        "classwise": load_pkl(ens_classwise_soft_nectar_binning_dir)
    },
}

In [10]:
# Loop through the calibration metrics and add them to the dataframe.
for cal_metric_name, cal_metric_dict in cal_metrics_cfg["global_cal_metrics"].items():
    for calibrator in ensemble_calibrator_dict:
        for split in ["cal"]:
            cal_type = cal_metric_dict['cal_type']
            if cal_type in ["classwise", "toplabel"]:
                print(f"CALIBRATOR:{calibrator}")
                if cal_type == 'classwise':
                    cal_info = cal_metric_dict['_fn'](
                        pixel_meters_dict=ensemble_calibrator_dict[calibrator]["classwise"][split],
                        return_dict=True
                    )
                else:
                    cal_info = cal_metric_dict['_fn'](
                        pixel_meters_dict=ensemble_calibrator_dict[calibrator]["top_label"][split],
                        return_dict=True
                    )
                print(f"Global Bin counts: (Total: {cal_info['bin_amounts'].sum()})")
                print(cal_info["bin_amounts"])
                print()
                print("Global Freqs:")
                print(cal_info["bin_freqs"])
                print()
                print("Global Confs:")
                print(cal_info["bin_confs"])
                print()
                print("Global Bin cal errors:")
                print(cal_info["bin_cal_errors"])
                print()
                print("SPLIT:{} | CALIBRATION METRIC:{} | LOSS:{}".format(
                    split, cal_metric_name, cal_info['cal_error']
                ))
                print()
                print()
            else:
                raise ValueError(f"Calibration type {cal_metric_dict['cal_type']} not recognized.")

CALIBRATOR:Uncalibrated
Global Bin counts: (Total: 172527.0)
tensor([10341.,  4479.,  4503.,  4504.,  4727.,  4858.,  5176.,  5512.,  5935.,
         6610.,  7482.,  8851., 10782., 16202., 72565.], dtype=torch.float64)

Global Freqs:
tensor([0.5306, 0.5490, 0.5587, 0.5564, 0.5949, 0.5955, 0.6148, 0.6386, 0.6492,
        0.6871, 0.6921, 0.7233, 0.7468, 0.7802, 0.8572], dtype=torch.float64)

Global Confs:
tensor([0.4797, 0.5498, 0.5835, 0.6166, 0.6501, 0.6833, 0.7169, 0.7501, 0.7835,
        0.8168, 0.8506, 0.8838, 0.9174, 0.9514, 0.9915], dtype=torch.float64)

Global Bin cal errors:
tensor([0.0509, 0.0008, 0.0248, 0.0602, 0.0552, 0.0878, 0.1022, 0.1115, 0.1343,
        0.1296, 0.1585, 0.1604, 0.1706, 0.1712, 0.1343], dtype=torch.float64)

SPLIT:cal | CALIBRATION METRIC:Edge-ECE | LOSS:0.12381500538003261


CALIBRATOR:NectarScaling
Global Bin counts: (Total: 172527.0)
tensor([11042.,  6230.,  7012.,  8415.,  9912., 11522., 12633., 13589., 14191.,
        14443., 15296., 16757., 16591., 1